In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd ..

/Users/alex/PETs-Experiment


In [4]:
import PETs_Experiment

In [5]:
loader = PETs_Experiment.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [6]:
df = loader.data

In [7]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [8]:
df

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,53.0,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,Male,78.0,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,Female,56.0,White,Graduate,Parther,42.400002,1,0,0.0,0.0,Q3,0
4,Female,42.0,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4186,Female,76.0,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4187,Female,80.0,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4188,Male,35.0,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [10]:
pm = PETs_Experiment.Processor_Manager(metadata)

In [11]:
pm